In [1]:
import os
if not os.path.exists("/scratch/sarthak"):
    os.makedirs("/scratch/sarthak")

os.environ["HF_HOME"] = "/scratch/sarthak/"

In [2]:
import torch
import torch.nn.functional as F
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModelForSeq2SeqLM
from typing import List, Tuple, Optional
import time
import numpy as np
import datasets

In [3]:
# # WMT16 EN-DE dataset
# def preprocess_function_ende(examples, tokenizer, args, train_args, prefix="translate English to German: "):
#     if train_args.debug:
#         all_text = examples['translation'][:2]
#     else:
#         all_text = examples['translation']
        
#     inputs = []
#     targets = []
#     for excerpt in all_text:
#         en_text = prefix + excerpt['en']
#         de_text = excerpt['de']

#         inputs.append(en_text)
#         targets.append(de_text)
            
#     padding = 'max_length'
#     model_inputs = tokenizer(
#         inputs,
#         max_length=args.source_max_length,
#         padding=padding,
#         truncation=True,
#         return_tensors="pt",
#     )
#     # Tokenize targets with the `text_target` keyword argument
#     labels = tokenizer(
#         text_target=targets,
#         max_length=args.train_target_max_length,
#         padding=padding,
#         truncation=True,
    #     return_tensors="pt",
    # )

    # if padding == "max_length":
    #             labels["input_ids"] = [
    #                 [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    #             ]

    # model_inputs["labels"] = labels["input_ids"]
    # model_inputs["decoder_attention_mask"] = labels["attention_mask"]
    # return model_inputs

en_gr_dataset = datasets.load_dataset('wmt16', 'de-en', split='validation')

# Base Speculative

In [4]:
class SpeculativeDecoder:
    def __init__(
        self,
        target_model_name = "google-t5/t5-3b",
        draft_model_name = "google-t5/t5-small",
        gamma = 4,
        temperature = 1.0
    ):
        self.gamma = gamma

        self.temperature = temperature

        self.tokenizer = T5Tokenizer.from_pretrained(target_model_name)
        self.target_model = AutoModelForSeq2SeqLM.from_pretrained(target_model_name, device_map='auto')

        self.draft_model = T5ForConditionalGeneration.from_pretrained(draft_model_name, device_map='auto')

        self.target_model.eval()
        self.draft_model.eval()

    def get_draft_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        gamma: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get draft logits for gamma tokens"""
        draft_tokens = []
        draft_probs = []
        current_decoder_ids = decoder_input_ids

        # Generate gamma tokens from the draft model
        for _ in range(gamma):
            with torch.no_grad():
                outputs = self.draft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=current_decoder_ids,
                    return_dict=True,
                )
                logits = outputs.logits[:, -1, :]  # Get logits for last position
                probs = logits
                # probs = F.softmax(logits, dim=-1)

                token_id = torch.argmax(probs, dim=-1)
                prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)

                draft_tokens.append(token_id.item())
                draft_probs.append(prob.item())

                # Update decoder inputs for next iteration
                current_decoder_ids = torch.cat(
                    [current_decoder_ids, token_id.view(1, 1)],
                    dim=1
                )

                if token_id.item() == self.tokenizer.eos_token_id:
                    break

        return draft_tokens, draft_probs, outputs.logits.squeeze(0)

    def get_target_probs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        draft_tokens: torch.Tensor,
    ) -> torch.Tensor:
        """Get target probabilities for the draft tokens in parallel."""
        with torch.no_grad():
            # Add draft tokens to decoder input
            full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)
            # print(input_ids_batched.shape, attention_mask_batched.shape, padded_decoder_ids.shape)
            # decoder_mask = torch.triu(
            #     torch.ones((full_decoder_ids.shape[1], full_decoder_ids.shape[1] + 1))
            # )
            # decoder_mask = decoder_mask[-(len(draft_tokens) + 1):, :-1]
            # decoder_mask = 1 - decoder_mask
            
            # the shapes that we want to see are:
            # torch.Size([11, 12]) torch.Size([1, 12])
            # torch.Size([11, 12]) torch.Size([11, 12])
            # torch.Size([11, 32128])
            # torch.Size([11, 32128]) torch.Size([11, 32128])

            # What im getting
            # torch.Size([1, 12]) torch.Size([1, 12])
            # torch.Size([1, 12]) torch.Size([1, 12])
            # torch.Size([1, 11, 32128])


            decoder_mask = torch.ones(full_decoder_ids.shape[1])
            decoder_mask = decoder_mask.unsqueeze(0)


            # print(decoder_mask.shape, full_decoder_ids.shape)

            # conver to a batched input
            # input_ids = input_ids.repeat(len(draft_tokens) + 1, 1)
            # attention_mask = attention_mask.repeat(len(draft_tokens) + 1, 1)
            # full_decoder_ids = full_decoder_ids.repeat(len(draft_tokens) + 1, 1)

            # print(decoder_mask.shape, full_decoder_ids.shape)


            outputs = self.target_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=full_decoder_ids,
                decoder_attention_mask=decoder_mask,
                return_dict=True
            )

            
            # dim_0_indices = torch.arange(len(draft_tokens) + 1)
            # dim_1_indices = torch.arange(len(draft_tokens) + 1) + full_decoder_ids.shape[1] - 1 - len(draft_tokens)
            # logits = outputs.logits[dim_0_indices, dim_1_indices, :]
            logits = outputs.logits[:, -(len(draft_tokens) + 1):, :]
            logits = logits.squeeze(0)

            # print(logits.shape)
            
            # Get probabilities for positions before each draft token
            # logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
            target_probs = F.softmax(logits, dim=-1)

            # print(target_probs.shape, target_probs.squeeze(0).shape)
            

            return target_probs.squeeze(0), logits

    def verify_tokens(
        self,
        target_probs: torch.Tensor,
        draft_tokens: torch.Tensor,
        draft_probs: torch.Tensor,
    ) -> int:
        """Determine number of accepted tokens"""
        # Get target probabilities for the draft tokens
        # get the probabilities of the tokens at the indices of the draft tokens
        target_probs_draft_tokens = torch.gather(target_probs, 1, draft_tokens.unsqueeze(0))
        # Calculate acceptance ratios
        acceptance_ratios = target_probs_draft_tokens / draft_probs.clamp(min=1e-10)

        # Sample uniform random numbers 
        random_nums = torch.rand_like(acceptance_ratios)
        acceptance_mask = random_nums <= acceptance_ratios

        num_accepted = (acceptance_mask.cumsum(dim=-1) == torch.arange(1, len(acceptance_ratios) + 1)).sum().item()

        return num_accepted

    def translate(
        self,
        source_text: str,
        max_length: int = 128
    ) -> str:
        """Translate source text using speculative decoding."""
        # Encode source text
        encoder_inputs = self.tokenizer(
            f"translate English to German: {source_text}",
            return_tensors="pt",
            padding=True
        )

        # Initialize with start token
        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]])

        output = self.target_model(
            input_ids=encoder_inputs.input_ids,
            attention_mask=encoder_inputs.attention_mask,
            decoder_input_ids=decoder_input_ids,
            return_dict=True
        )

        probs = output.logits[:, -1, :]
                    
        probs = F.softmax(probs / (self.temperature + 1e-13), dim=-1)
        token_id = torch.multinomial(probs, num_samples=1)

        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id, token_id.item()]])

        total_tokens = 0
        accepted_tokens = 0

        while decoder_input_ids.shape[1] < max_length:
            # Get draft tokens autoregressively
            draft_tokens, draft_probs, draft_logits = self.get_draft_logits(
                encoder_inputs.input_ids,
                encoder_inputs.attention_mask,
                decoder_input_ids,
                self.gamma
            )

            draft_tokens = torch.tensor(draft_tokens)
            draft_probs = torch.tensor(draft_probs)

            # softmax the draft probs
            draft_probs = F.softmax(draft_probs / (self.temperature + 1e-13), dim=-1)

            if len(draft_tokens) == 0:
                raise ValueError("Draft tokens not generated.")

            # Get target probabilities in parallel
            # start = time.time()
            target_probs, target_logits = self.get_target_probs(
                encoder_inputs.input_ids,
                encoder_inputs.attention_mask,
                decoder_input_ids,
                draft_tokens
            )
            # target probs are the logits but with softmax applied

            # Verify tokens
            n_accepted = self.verify_tokens(target_probs, draft_tokens, draft_probs)
            # Accept verified tokens
            if n_accepted > 0:
                decoder_input_ids = torch.cat([
                    decoder_input_ids,
                    draft_tokens[:n_accepted].unsqueeze(0)
                ], dim=1)
                
            with torch.no_grad():
                # n_rejected = self.gamma - n_accepted
                n_rejected = len(draft_tokens) - n_accepted 
                total_tokens += len(draft_tokens)
                accepted_tokens += n_accepted

                if n_rejected > 0:
                    probs = target_logits[-n_rejected, :] #- draft_logits[1-n_rejected, :]
                else:
                    probs = target_logits[-1, :]
                    
                probs = F.softmax(probs / (self.temperature + 1e-13), dim=-1)
                # probs /= max(self.temperature, 1e-13)
                # probs_max = torch.where(probs > 0, probs, torch.zeros_like(probs))
                # probs_max_sum = torch.sum(probs_max)
                # probs = probs_max / max(probs_max_sum, 1e-13)
                
                token_id = torch.multinomial(probs, num_samples=1).unsqueeze(0)

                decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)

            # Check for end of sequence
            if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                break

            # # or if a full stop is generated
            # if decoder_input_ids[0][-1].item() == self.tokenizer.convert_tokens_to_ids('.'):
            #     break

        # Decode translation
        translation = self.tokenizer.decode(
            decoder_input_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        perc_accepted = accepted_tokens / total_tokens * 100
        return translation, perc_accepted

In [5]:
# just translate using the big model
big_model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-3b", device_map='auto')

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-3b")

source_text = "translate English to German: Formula One cars are the world's fastest regulated road-course racing cars, owing to very high cornering speeds achieved by generating large amounts of aerodynamic downforce, much of which is generated by front and rear wings."
# source_text = "translate English to German: India is also reportedly hoping for a deal on defence collaboration between the two nations."
start = time.time()

input_ids = tokenizer(source_text, return_tensors="pt").input_ids.to(big_model.device)
decoder_input_ids = torch.tensor([[tokenizer.pad_token_id]], device=big_model.device)  # Start with PAD token
eos_token_id = tokenizer.eos_token_id

output_tokens = []
while True:
    # Predict the next token
    outputs = big_model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
    next_token_logits = outputs.logits[:, -1, :]  # Get logits for the last token
    next_token_id = torch.argmax(next_token_logits, dim=-1)  # Greedy decoding

    # Append the generated token to the decoder input
    decoder_input_ids = torch.cat([decoder_input_ids, next_token_id.unsqueeze(0)], dim=-1)
    output_tokens.append(next_token_id.item())

    # Break the loop if the EOS token is generated
    if next_token_id.item() == eos_token_id:
        break

big_translation = tokenizer.decode(output_tokens, skip_special_tokens=True)

# Print the results
print("Big model translation: ", big_translation)
print("Time taken: ", time.time() - start)


xonsh: For full traceback set: $XONSH_SHOW_TRACEBACK = True
NameError: name 'AutoModelForSeq2SeqLM' is not defined


In [4]:
speculative_decoder1 = SpeculativeDecoder(gamma=6, temperature=0.0)
speculative_decoder2 = SpeculativeDecoder(gamma=6, temperature=0.5)
speculative_decoder3 = SpeculativeDecoder(gamma=6, temperature=1.0)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
source_text = "Formula One cars are the world's fastest regulated road-course racing cars, owing to very high cornering speeds achieved by generating large amounts of aerodynamic downforce, much of which is generated by front and rear wings."
# source_text = "India is also reportedly hoping for a deal on defence collaboration between the two nations."
speculative_decoder1.translate(source_text)
start = time.time()
speculative_translation, pc = speculative_decoder1.translate(source_text)
end = time.time()
print(f"Speculative translation: {speculative_translation}")
print(f"Percentage tokens accepted: {pc:.2f}%")
print(f"Time taken: {end - start:.4f} seconds")

Speculative translation: Die For 1erFahrzeug sind die schnellstener reguliterenn Renwagen der, da sehr hohekeln Geschwindigkeiten erreicht, die die Erzeug von  Mengen aerocher Abungskrafterzeug, die von vorne hinten von vorne hinterngelb Flügel erzeugt werden
Percentage tokens accepted: 19.39%
Time taken: 6.5364 seconds


In [6]:
# source_text = "India is also reportedly hoping for a deal on defence collaboration between the two nations."
speculative_decoder2.translate(source_text)
start = time.time()
speculative_translation, pc = speculative_decoder2.translate(source_text)
end = time.time()
print(f"Speculative translation: {speculative_translation}")
print(f"Percentage tokens accepted: {pc:.2f}%")
print(f"Time taken: {end - start:.4f} seconds")

Speculative translation: Die Formel 1 das amsten regulitierte Renauto der Welt aufgrund der sehr hohen Kurvengeschwindigkeiten, die durch die Erzeugung vongroßen Menge von aerodynamischer Abschwächung erzielt werden die meisten durch Vorund undflügel erzeugt werden.
Percentage tokens accepted: 50.60%
Time taken: 3.3020 seconds


In [7]:
# source_text = "India is also reportedly hoping for a deal on defence collaboration between the two nations."
speculative_decoder3.translate(source_text)
start = time.time()
speculative_translation, pc = speculative_decoder3.translate(source_text)
end = time.time()
print(f"Speculative translation: {speculative_translation}")
print(f"Percentage tokens accepted: {pc:.2f}%")
print(f"Time taken: {end - start:.4f} seconds")

Speculative translation: Die Formel 1 ist Spitzenklasse, die weltweit am schnellsten reguliert ist, aufgrund der sehr hohen Kurvengeschwindigkeiten, die durch einen hohen aerodynamischen Abstützen erreicht werden, der größtenteil durch Vor- und Hinterflügel erzeugt wird
Percentage tokens accepted: 84.21%
Time taken: 2.3587 seconds


# Normal

In [5]:
class NormalDecoder:
    def __init__(
        self,
        model_name: str = "google-t5/t5-3b",
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
    ):
        self.device = device

        # Initialize tokenizer and model
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        # self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto')
        self.model.eval()

    def get_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor
    ) -> torch.Tensor:
        """Get logits from model for the last token."""
        with torch.no_grad():
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                return_dict=True
            )
            return outputs.logits[:, -1, :]

    def sample_token(self, logits: torch.Tensor, temperature: float = 1.0) -> Tuple[torch.Tensor, torch.Tensor]:
        """Sample a token from logits using temperature sampling."""
        if temperature == 0:
            # Greedy sampling
            token_id = torch.argmax(logits, dim=-1)
            prob = torch.ones_like(token_id, dtype=torch.float)
        else:
            # Temperature sampling
            probs = F.softmax(logits / temperature, dim=-1)
            token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
            prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)
        return token_id, prob

    def translate(
        self,
        source_text: str,
        max_length: int = 128,
        temperature: float = 0.7
    ) -> str:
        """Translate source text using the normal T5 model."""
        # Encode source text
        encoder_inputs = self.tokenizer(
            f"translate English to German: {source_text}",
            return_tensors="pt",
            padding=True
        ).to(self.device)

        # Initialize decoder input with start token
        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)

        while decoder_input_ids.shape[1] < max_length:
            # Generate logits for the next token
            logits = self.get_logits(
                encoder_inputs.input_ids,
                encoder_inputs.attention_mask,
                decoder_input_ids
            )

            # Sample a token
            token_id, _ = self.sample_token(logits, temperature)

            # Add token to the decoder input
            decoder_input_ids = torch.cat(
                [decoder_input_ids, token_id.view(1, 1)],
                dim=1
            )

            # Break if end token is generated
            if token_id.item() == self.tokenizer.eos_token_id:
                break

        # Decode and return translation
        translation = self.tokenizer.decode(
            decoder_input_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        return translation


In [7]:
# speculative_decoder = SpeculativeDecoder()
# normal_decoder = NormalDecoder()

# source_text = "He said Lamb made the fateful 911 call sometime after that." # spec does not work

# speculative_decoder.translate(source_text)
# start = time.time()
# speculative_translation, pc = speculative_decoder.translate(source_text)
# end = time.time()
# print(f"Speculative translation: {speculative_translation}")
# print(f"Percentage tokens accepted: {pc:.2f}%")
# print(f"Time taken: {end - start:.4f} seconds")

# start = time.time()
# normal_translation = normal_decoder.translate(source_text)
# end = time.time()
# print(f"Normal translation: {normal_translation}")
# print(f"Time taken: {end - start:.4f} seconds")

In [7]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=7, temperature=1.0)
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['translation'][:30]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    # if normal_time - spec_time > -0.1:
    spec_total_time += spec_time
    normal_total_time += normal_time
    total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

  3%|▎         | 1/30 [00:03<01:31,  3.16s/it]

Source: India and Japan prime ministers meet in Tokyo
Normal Translation: Indiens und Japans Premierminister treffen sich in Tokio
Time taken: 1.42 seconds
Speculative Translation: Indien und Japans Premierminister treffen sich Tokio
Time taken: 1.73 seconds
Percentage tokens accepted: 68.75%
Target: Die Premierminister Indiens und Japans trafen sich in Tokio.


  7%|▋         | 2/30 [00:11<02:51,  6.12s/it]

Source: India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.
Normal Translation: Der neue indische Ministerpräsident Narendra Modi trifft in Tokio auf seinem ersten großen Auslandsbesuch nach dem Wahlsieg im Mai mit seinem japanischen Amtskollegen Shinzo Abe, um über die wirtschaftlichen und Sicherheitsbeziehungen zu sprechen.
Time taken: 5.71 seconds
Speculative Translation: In Indiens neuer Premierminister Narendra Modi trifft sein japanisches Amtkollege Shinzo Ab, um über wirtschaftliche und Sicherheitsbeziehungen zu diskutieren, und das in seinemersten großen Auslandsbesuch seit dem Wahlsieg im Mai.
Time taken: 2.47 seconds
Percentage tokens accepted: 81.97%
Target: Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Tok

 10%|█         | 3/30 [00:16<02:27,  5.48s/it]

...

Source: Indeed, the best way to understand what Ellison, Lewis and the cosponsors of their legislation are proposing is as a reconnection with a very American idea.
Normal Translation: Tatsächlich ist das, was Ellison, Lewis und die Mitbegründer ihrer Gesetzesvorlage vorschlagen, beispielhaft als eine Wiederanbindung an eine höchst amerikanische Idee zu verstehen.
Time taken: 4.22 seconds
Speculative Translation: Tatsächlich ist die beste Art, verstehen zu können was Ellison, Lewis und die Mittragenden ihrer Gesetzgebung vorschlagen, eine Verknüpfung mit einer sehr amerikanischen Idee.
Time taken: 2.05 seconds
Percentage tokens accepted: 76.47%
Target: Tatsächlich ist die beste Art und Weise zum Verständnis dessen, was Ellison, Lewis und die weiteren Sponsoren ihrer Gesetzesvorlage vorschlagen, die Verbindung zurück zu einer sehr amerikanischen Idee.


 97%|█████████▋| 29/30 [02:28<00:07,  7.37s/it]

Source: Despite the battering that unions have taken in recent years - in Wisconsin, Michigan and states across the country - Americans once encouraged countries around the world to embrace, extend and respect labor rights.
Normal Translation: Trotz des heftigen Schlags, den Gewerkschaften in den letzten Jahren erlitten haben - in Wisconsin, Michigan und in ganzen Bundesstaaten - ermutigten Amerikaner einst Länder auf der ganzen Welt, Arbeitsrechte zu akzeptieren, auszubauen und zu respektieren.
Time taken: 7.03 seconds
Speculative Translation: Trotz der erschütternden Anschläge, die die Gewerkschaften in denletzten Jahren - in Wisconsin, Michigan und in Bundesstaaten überall im Land - ergriffen haben,ermunterten die Amerikaner Länder auf deren Welt, ihre Arbeitsrechte zu berücksichtig, auszuweiten und achten
Time taken: 3.41 seconds
Percentage tokens accepted: 69.77%
Target: Trotz der Rückschläge, denen die Gewerkschaften in den vergangenen Jahren ausgesetzt waren - in Wisconsin, Mich

100%|██████████| 30/30 [02:35<00:00,  5.18s/it]

Source: There was a time, within the living memory of millions of Americans, when this country championed democracy, freedom of speech, freedom of the press and the right to organize in the same breath.
Normal Translation: Es gab eine Zeit, die Millionen von Amerikanern noch in frischer Erinnerung hat, als dieses Land gleichzeitig Demokratie, Meinungsfreiheit, Pressefreiheit und das Recht auf Vereinigungsfreiheit verteidigte.
Time taken: 4.68 seconds
Speculative Translation: Es gab eine Zeit, in der Millionen von Amerikanern leben, als dieses Land Demokrati, Redfreiheit, Pressefreiheit und das Recht auf Organisation im gleichen Atemzug unterstützte
Time taken: 1.81 seconds
Percentage tokens accepted: 80.00%
Target: Es gab eine Zeit, an die sich Millionen von Amerikanern noch erinnern, als dieses Land Demokratie, Redefreiheit, Pressefreiheit und das Vereinigungsrecht in einem Atemzug nannte.

Average time taken for normal decoding: 3.31 seconds
Average time taken for speculative decodin

In [6]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=7, temperature=1)
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['translation'][:30]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    # if normal_time - spec_time > -0.1:
    spec_total_time += spec_time
    normal_total_time += normal_time
    total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
  3%|▎         | 1/30 [00:03<01:47,  3.71s/it]

Source: India and Japan prime ministers meet in Tokyo
Normal Translation: Indien und Japan treffen sich in Tokio
Time taken: 0.96 seconds
Speculative Translation: Indien und Japans Premierminister treffen sich Tokio
Time taken: 2.75 seconds
Percentage tokens accepted: 68.75%
Target: Die Premierminister Indiens und Japans trafen sich in Tokio.


  7%|▋         | 2/30 [00:14<03:46,  8.08s/it]

Source: India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.
Normal Translation: Indiens neuer Ministerpräsident Narendra Modi trifft in Tokio auf seinem ersten großen Auslandsbesuch seit dem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe, um wirtschaftliche und Sicherheitsbeziehungen zu besprechen.
Time taken: 5.30 seconds
Speculative Translation: Die deutsche Regierung, die Präsidentin UNF hat, hat sich mit den Präsidenten der Europäische Union in der Europäische Union angehört, die sich mit der Europäische Transportunion befassen. Glaubwürdigkeit die Europäische Union, dieVielen zu erm glichen, die Europäische Union zugen hat, und dieEuropäische Unionhat sich mit der Europäische Union befass, die Arbeit der Europäische Union zuerkann.
Time taken: 5.84 seconds
Percentage tokens accepted: 54.17%
Target: Indiens neuer Premierm

 10%|█         | 3/30 [00:19<02:52,  6.38s/it]

...

Source: Indeed, the best way to understand what Ellison, Lewis and the cosponsors of their legislation are proposing is as a reconnection with a very American idea.
Normal Translation: Tatsächlich lässt sich das, was Ellison, Lewis und die Mitentwickler ihrer Gesetze vorschlagen, am besten als eine Wiederherstellung einer sehr amerikanischen Idee verstehen.
Time taken: 3.73 seconds
Speculative Translation: Tatsächlich ist die beste Art, verstehen zu können was Ellison, Lewis und die Mitprofessoren ihrer Geetzgebung vorschlagen, eine Verknüpfung mit einer sehr amerikanischen Idee.30.
Time taken: 2.22 seconds
Percentage tokens accepted: 76.36%
Target: Tatsächlich ist die beste Art und Weise zum Verständnis dessen, was Ellison, Lewis und die weiteren Sponsoren ihrer Gesetzesvorlage vorschlagen, die Verbindung zurück zu einer sehr amerikanischen Idee.


 97%|█████████▋| 29/30 [02:33<00:07,  7.21s/it]

Source: Despite the battering that unions have taken in recent years - in Wisconsin, Michigan and states across the country - Americans once encouraged countries around the world to embrace, extend and respect labor rights.
Normal Translation: Trotz der Schläge, die Gewerkschaften in den letzten Jahren - in Wisconsin, Michigan und in den Bundesstaaten - erlitten haben, haben die Amerikaner einst Länder weltweit ermutigt, die Arbeitnehmerrechte anzuerkennen, zu erweitern und zu respektieren.
Time taken: 6.55 seconds
Speculative Translation: Trotz der erschütternden Behandlungen die Gewerkschaften den letzten Jahren- in Wisconsin, Michigan und überall im Land - ermutigten die Amerikaner einst Länder auf der ganzen Welt, ihre Arbeitsrecht zu berücksichtig, auszuweiten zu respektieren sie zureden und zu reieren.
Time taken: 3.90 seconds
Percentage tokens accepted: 63.00%
Target: Trotz der Rückschläge, denen die Gewerkschaften in den vergangenen Jahren ausgesetzt waren - in Wisconsin, Michi

100%|██████████| 30/30 [02:39<00:00,  5.32s/it]

Source: There was a time, within the living memory of millions of Americans, when this country championed democracy, freedom of speech, freedom of the press and the right to organize in the same breath.
Normal Translation: Es gab eine Zeit, die Millionen von Amerikanern noch gut in Erinnerung hat, als dieses Land mit dem gleichen Atem Demokratie, Redefreiheit, Pressefreiheit und das Recht auf Organisation verteidigte.
Time taken: 4.71 seconds
Speculative Translation: Einer Zeit, in der Millionen von Amerikanern leben, war dieses Land für Demokrati, Redefreiheit, Pressefreiheit und das auf demselben Atemweg zu vertretende Recht.
Time taken: 1.79 seconds
Percentage tokens accepted: 84.44%
Target: Es gab eine Zeit, an die sich Millionen von Amerikanern noch erinnern, als dieses Land Demokratie, Redefreiheit, Pressefreiheit und das Vereinigungsrecht in einem Atemzug nannte.

Average time taken for normal decoding: 3.31 seconds
Average time taken for speculative decoding: 2.01 seconds
Avera

In [6]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=7, temperature=0.0)
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['translation'][:30]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    # if normal_time - spec_time > -0.1:
    spec_total_time += spec_time
    normal_total_time += normal_time
    total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
  3%|▎         | 1/30 [00:04<02:17,  4.73s/it]

Source: India and Japan prime ministers meet in Tokyo
Normal Translation: Indiens und Japans Ministerpräsidenten treffen sich in Tokio
Time taken: 1.40 seconds
Speculative Translation: Indien Japan und Ministerpräsident treffen in Toki
Time taken: 3.33 seconds
Percentage tokens accepted: 18.75%
Target: Die Premierminister Indiens und Japans trafen sich in Tokio.


  7%|▋         | 2/30 [00:15<03:57,  8.47s/it]

Source: India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.
Normal Translation: Der neue indische Ministerpräsident Narendra Modi trifft sich in Tokio mit seinem japanischen Amtskollegen Shinzo Abe, um wirtschaftliche und Sicherheitsbeziehungen zu erörtern, und zwar auf seinem ersten großen Auslandsbesuch seit dem Wahlsieg im Mai.
Time taken: 6.22 seconds
Speculative Translation: Der neuedines Premier Narendrai trifft japanse Premierminister Shi Abe Toki, um seinenersten großenbesuch seit Wahlsieg Mai inkio zu disktieren.
Time taken: 4.86 seconds
Percentage tokens accepted: 17.97%
Target: Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Toko, um wirtschaftliche und sicherheitspolitische Beziehungen zu besprechen.


 10%|█         | 3/30 [00:22<03:25,  7.61s/it]

...

Source: Indeed, the best way to understand what Ellison, Lewis and the cosponsors of their legislation are proposing is as a reconnection with a very American idea.
Normal Translation: In der Tat wird das, was Ellison, Lewis und die anderen Unterstützer ihrer Gesetze vorschlagen, am besten als Rückconnection zu einer sehr amerikanischen Idee verstanden.
Time taken: 4.21 seconds
Speculative Translation: Tatslich ist beste Verständnissweise was Ellis, Lewis die Verechterr ihre Gesetz vorschlagen als einekoppel mit einer sehramerikanischen Idee
Time taken: 4.49 seconds
Percentage tokens accepted: 17.65%
Target: Tatsächlich ist die beste Art und Weise zum Verständnis dessen, was Ellison, Lewis und die weiteren Sponsoren ihrer Gesetzesvorlage vorschlagen, die Verbindung zurück zu einer sehr amerikanischen Idee.


 97%|█████████▋| 29/30 [03:34<00:11, 11.85s/it]

Source: Despite the battering that unions have taken in recent years - in Wisconsin, Michigan and states across the country - Americans once encouraged countries around the world to embrace, extend and respect labor rights.
Normal Translation: Trotz der Zerrüttung, die die Gewerkschaften in den letzten Jahren – in Wisconsin, Michigan und in allen Bundesstaaten – erlitten haben, ermutigten Amerikaner einst die Länder rund um die Welt, Arbeitsrechte zu begrüßen, auszubauen und zu respektieren.
Time taken: 7.93 seconds
Speculative Translation: Trotz der nüchtendeneinander reisierten Gewerkschaft in denletzten Jahren  in Wisconsin Michigan und allen Staat des Landeserbten mutierten Amerner eintä dazu, Länder der Welt daran teil, diee Recht der Arbeitnehmer akeptieren ausweit und zu achten
Time taken: 9.16 seconds
Percentage tokens accepted: 16.91%
Target: Trotz der Rückschläge, denen die Gewerkschaften in den vergangenen Jahren ausgesetzt waren - in Wisconsin, Michigan und anderen Staaten 

100%|██████████| 30/30 [03:47<00:00,  7.58s/it]

Source: There was a time, within the living memory of millions of Americans, when this country championed democracy, freedom of speech, freedom of the press and the right to organize in the same breath.
Normal Translation: Es gab eine Zeit, die noch im lebendigen Gedächtnis von Millionen von Amerikanern gelebt wird, als dieses Land in einem Atemzug für Demokratie, Redefreiheit, Pressefreiheit und das Recht auf Organisation eintrat.
Time taken: 6.26 seconds
Speculative Translation: Es gab Zeiten die in lebendige Gedächnis Millionen vonern Amernernreichen standen als diese Land Demoratie Rede, Pressfreiheit und Recht auf im gleich Atemzugverein vertte
Time taken: 6.62 seconds
Percentage tokens accepted: 17.33%
Target: Es gab eine Zeit, an die sich Millionen von Amerikanern noch erinnern, als dieses Land Demokratie, Redefreiheit, Pressefreiheit und das Vereinigungsrecht in einem Atemzug nannte.

Average time taken for normal decoding: 3.69 seconds
Average time taken for speculative decodi

In [6]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=7, temperature=0.8)
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['translation'][:30]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    # if normal_time - spec_time > -0.1:
    spec_total_time += spec_time
    normal_total_time += normal_time
    total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
  3%|▎         | 1/30 [00:04<01:57,  4.05s/it]

Source: India and Japan prime ministers meet in Tokyo
Normal Translation: Indien und Japan unterhalten sich in Tokio
Time taken: 1.17 seconds
Speculative Translation: Indien und Japans Premierminister treffen sich Tokio
Time taken: 2.88 seconds
Percentage tokens accepted: 68.75%
Target: Die Premierminister Indiens und Japans trafen sich in Tokio.


  7%|▋         | 2/30 [00:14<03:40,  7.89s/it]

Source: India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.
Normal Translation: Der neue indische Premierminister Narendra Modi trifft sich heute mit seinem japanischen Kollegen Shinzo Abe in Tokio, um über wirtschaftliche und sicherheitspolitische Beziehungen zu sprechen. Dies ist sein erster größerer Auslandsbesuch seit dem Wahlsieg im Mai.
Time taken: 6.67 seconds
Speculative Translation: Der neuediente Ministerpräsident Narendra Mod trifft sein japanische Amtskollege Shinzo Abe in Tokio um über wirtschaftliche und Sicherheitsbeziehungen zu diskutieren,während erseinen erstenwichtigen Außenbesuch seit dem Sieg der Wahlen im Mai geführt hat
Time taken: 3.91 seconds
Percentage tokens accepted: 63.10%
Target: Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen j

 10%|█         | 3/30 [00:19<02:53,  6.42s/it]

...

Source: Indeed, the best way to understand what Ellison, Lewis and the cosponsors of their legislation are proposing is as a reconnection with a very American idea.
Normal Translation: Tatsächlich verstehen wir das, was Ellison, Lewis und die Mitbegründer ihres Gesetzesentwurfs vorschlagen, am besten als eine Wiederherstellung einer sehr amerikanischen Idee.
Time taken: 3.97 seconds
Speculative Translation: Tatsächlich ist die beste Art, verstehen zu können was Ellison, Lewis und die Mitsponoren ihrer Gesetzgebung vorschlagen, eine Verknüpfung mit einer sehr amerikanischen Idee.
Time taken: 2.30 seconds
Percentage tokens accepted: 68.97%
Target: Tatsächlich ist die beste Art und Weise zum Verständnis dessen, was Ellison, Lewis und die weiteren Sponsoren ihrer Gesetzesvorlage vorschlagen, die Verbindung zurück zu einer sehr amerikanischen Idee.


 97%|█████████▋| 29/30 [02:26<00:07,  7.03s/it]

Source: Despite the battering that unions have taken in recent years - in Wisconsin, Michigan and states across the country - Americans once encouraged countries around the world to embrace, extend and respect labor rights.
Normal Translation: Trotz der Schläge, die die Gewerkschaften in den letzten Jahren - in Wisconsin, Michigan und anderen Bundesstaaten - erlitten haben, ermutigten Amerikaner einst Länder auf der ganzen Welt, ihre Arbeitsrechte zu übernehmen, auszubauen und zu achten.
Time taken: 6.27 seconds
Speculative Translation: Trotz der erschütternden Lesezeiten, die die Gewerkschaften in den letzten Jahren - in Wisconsin, Michigan und überall im Land  - ergriffen haben, ermutigten die Amikaner einst Länder auf der ganzen Welt, ihre Arbeitsrechte zu berücksichtig, auszuweit und zu respektieren
Time taken: 3.27 seconds
Percentage tokens accepted: 79.76%
Target: Trotz der Rückschläge, denen die Gewerkschaften in den vergangenen Jahren ausgesetzt waren - in Wisconsin, Michigan u

100%|██████████| 30/30 [02:33<00:00,  5.12s/it]

Source: There was a time, within the living memory of millions of Americans, when this country championed democracy, freedom of speech, freedom of the press and the right to organize in the same breath.
Normal Translation: Es gab eine Zeit, die Millionen von Amerikanern noch gut im Gedächtnis hat, als dieses Land Demokratie, Redefreiheit, Pressefreiheit und das Recht auf Organisationsfreiheit in einem Atemzug verteidigte.
Time taken: 5.39 seconds
Speculative Translation: Es gab eine Zeit, in der Millionen von Amerikanern leben, als dieses Land Demokrati, Redfreiheit, Pressfreiheit und das Recht, sich in gleicher Weise zu organisieren, für sich einsetzt.
Time taken: 2.25 seconds
Percentage tokens accepted: 72.22%
Target: Es gab eine Zeit, an die sich Millionen von Amerikanern noch erinnern, als dieses Land Demokratie, Redefreiheit, Pressefreiheit und das Vereinigungsrecht in einem Atemzug nannte.

Average time taken for normal decoding: 3.06 seconds
Average time taken for speculative de

# Online Spec

In [ ]:
class OnlineSpeculativeDecoder:
    def __init__(
        self,
        target_model_name = "google-t5/t5-large",
        draft_model_name = "google-t5/t5-small",
        device = "cuda" if torch.cuda.is_available() else "cpu",
        gamma = 4,
        update_interval=2,  # Update draft model after every `update_interval` iterations
        buffer_size_threshold=2,  # Buffer size threshold for updates
        time_threshold=2,  # Time threshold (in seconds) for updates
    ):
        self.device = device
        self.gamma = gamma
        self.update_interval = update_interval
        self.buffer_size_threshold = buffer_size_threshold
        self.time_threshold = time_threshold
        self.last_update_time = time.time()  # Track last update time

        self.tokenizer = T5Tokenizer.from_pretrained(target_model_name)
        self.target_model = T5ForConditionalGeneration.from_pretrained(target_model_name).to(device)
        self.draft_model = T5ForConditionalGeneration.from_pretrained(draft_model_name).to(device)

        self.target_model.eval()
        self.draft_model.eval()

        # Buffers for storing token proposals and updates
        self.replay_buffer = []
        self.temp_buffer = []  # Temporary buffer for a single request

        # Counter for iteration tracking
        self.iteration_count = 0

    def get_draft_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        gamma: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get draft logits for gamma tokens"""
        draft_tokens = []
        draft_probs = []
        current_decoder_ids = decoder_input_ids

        # Generate gamma tokens from the draft model
        for _ in range(gamma):
            with torch.no_grad():
                outputs = self.draft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=current_decoder_ids,
                    return_dict=True
                )
                logits = outputs.logits[:, -1, :]  # Get logits for last position
                probs = F.softmax(logits, dim=-1)

                # Sample token
                token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
                prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)

                draft_tokens.append(token_id.item())
                draft_probs.append(prob.item())

                # Update decoder inputs for next iteration
                current_decoder_ids = torch.cat(
                    [current_decoder_ids, token_id.view(1, 1)],
                    dim=1
                )

                if token_id.item() == self.tokenizer.eos_token_id:
                    break

        return draft_tokens, draft_probs, current_decoder_ids, outputs.logits

    def get_target_probs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        draft_tokens: torch.Tensor
    ) -> torch.Tensor:
        """Get target probabilities for the draft tokens in parallel."""
        with torch.no_grad():
            # Add draft tokens to decoder input
            full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)

            outputs = self.target_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=full_decoder_ids,
                return_dict=True
            )

            # Get probabilities for positions before each draft token
            logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
            target_probs = F.softmax(logits, dim=-1)

            return target_probs.squeeze(0), outputs.logits
        
    def get_logits(self, model, input_ids, attention_mask):
        return model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        ).logits

    def verify_tokens(
        self,
        target_probs: torch.Tensor,
        draft_tokens: torch.Tensor,
        draft_probs: torch.Tensor,
    ) -> int:
        """Determine number of accepted tokens"""
        # Get target probabilities for the draft tokens
        target_probs_draft_tokens = target_probs.gather(
            -1,
            draft_tokens.unsqueeze(-1)
        ).squeeze(-1)

        # Calculate acceptance ratios
        acceptance_ratios = target_probs_draft_tokens / draft_probs

        # Sample uniform random numbers
        random_nums = torch.rand_like(acceptance_ratios)

        # Find number of accepted tokens
        # Accept if random number < min(1, target_prob / draft_prob)
        accepts = random_nums < torch.minimum(
            torch.ones_like(acceptance_ratios),
            acceptance_ratios
        )

        # Find first rejection
        try:
            n_accepted = torch.where(~accepts)[0][0].item()
        except:
            n_accepted = len(accepts)

        return n_accepted

        # accepted_tokens = []
        # for i in range(len(draft_tokens)):
        #     if target_probs[i] / draft_probs[i] > torch.rand(1).item():
        #         accepted_tokens.append(draft_tokens[i])
        #     else:
        #         break # Stop if token is not accepted

        # return len(accepted_tokens)
    
    # TODO: verify this, might need to do some window size thing
    def update_draft_model(self):
        """Update draft model with the replay buffer."""
        if len(self.replay_buffer) == 0:
            return

        # Get draft tokens, draft and target probabilities from the replay buffer
        # draft_tokens = torch.tensor([x[0] for x in self.replay_buffer], device=self.device)
        # print(self.replay_buffer[0][0])
        # draft_probs = self.replay_buffer[:, 0]
        # target_probs = self.replay_buffer[:, 1]
        draft_probs = torch.stack([x[0][0] for x in self.replay_buffer], dim=0)
        target_probs = torch.stack([x[1][0] for x in self.replay_buffer], dim=0)

        self.draft_model.train()

        # for param in self.draft_model.parameters():
        #     print(param.requires_grad)


        # criterion = torch.nn.CrossEntropyLoss()
        # print(draft_probs.shape, target_probs.shape)
        loss = self.soft_cross_entropy(draft_probs, target_probs)
        print("Loss grad_fn:", loss.grad_fn)
        print("Draft probs grad_fn:", draft_probs.grad_fn)
        print("Target probs grad_fn:", target_probs.grad_fn)

        loss.backward()

        self.draft_model.eval()

        # Clear the replay buffer
        self.replay_buffer = []

    def soft_cross_entropy(self, predicts, targets, padding_mask=None):
        predict_log_prob = torch.nn.functional.log_softmax(predicts, dim=-1)
        targets_prob = torch.nn.functional.softmax(targets, dim=-1)
        entropy = -targets_prob * predict_log_prob
        # expand_mask = padding_mask.unsqueeze(-1).expand_as(entropy)
        # entropy.masked_fill_(expand_mask, 0)
        # mean_entropy = entropy.sum() / (~padding_mask).sum()
        return entropy

    def translate(
        self,
        source_text: str,
        max_length: int = 128
    ) -> str:
        """Translate source text using speculative decoding."""
        # Encode source text
        encoder_inputs = self.tokenizer(
            f"translate English to German: {source_text}",
            return_tensors="pt",
            padding=True
        ).to(self.device)

        # Initialize with start token
        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)

        self.iteration_count = 0
        self.replay_buffer = []

        while decoder_input_ids.shape[1] < max_length:
            self.temp_buffer = []

            while decoder_input_ids.shape[1] < max_length:
                # Get draft tokens autoregressively
                # print("Encoder Inputs", encoder_inputs.input_ids.shape)
                draft_tokens, draft_probs, proposed_decoder_ids, draft_logits = self.get_draft_logits(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    self.gamma
                )

                draft_tokens = torch.tensor(draft_tokens, device=self.device)
                draft_probs = torch.tensor(draft_probs, device=self.device)

                if len(draft_tokens) == 0:
                    break

                # Get target probabilities in parallel
                target_probs, target_logits = self.get_target_probs(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    draft_tokens
                )
                # print(draft_logits.shape, target_logits.shape)

                # Verify tokens
                n_accepted = self.verify_tokens(target_probs, draft_tokens, draft_probs)

                # Accept verified tokens
                if n_accepted > 0:
                    decoder_input_ids = torch.cat([
                        decoder_input_ids,
                        draft_tokens[:n_accepted].unsqueeze(0)
                    ], dim=1)

                # # If rejection or no acceptance, sample one token from target
                # if n_accepted < len(draft_tokens):
                #     with torch.no_grad():
                #         outputs = self.target_model(
                #             input_ids=encoder_inputs.input_ids,
                #             attention_mask=encoder_inputs.attention_mask,
                #             decoder_input_ids=decoder_input_ids,
                #             return_dict=True
                #         )
                #         logits = outputs.logits[:, -1, :]
                #         probs = F.softmax(logits, dim=-1)
                #         token_id = torch.multinomial(probs, num_samples=1)
                #         decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)

                with torch.no_grad():
                    # print(target_logits.shape, draft_logits.shape)
                    probs = target_logits[:, -1, :] #- draft_logits[:, -1, :]
                    probs = F.softmax(probs, dim=-1)
                    token_id = torch.multinomial(probs, num_samples=1)
                    # print(probs.shape, token_id.shape)
                    decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)

                # Check for end of sequence
                if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                    break
                
                # TODO: Update buffer with draft and target logits of the first rejected token, verify implementation
                # rejected_tokens = draft_tokens[n_accepted]
                if n_accepted < len(draft_tokens):
                    # rejected_prob_draft = draft_logits[:, n_accepted, :]
                    # rejected_prob_target = target_logits[:, n_accepted, :]

                    self.temp_buffer.append((draft_logits[:, -1, :], target_logits[:, -1, :]))

                # Check for end of sequence
                if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                    break

            self.replay_buffer.extend(self.temp_buffer)
            self.iteration_count += 1

            if self.iteration_count % self.update_interval == 0:
                self.update_draft_model()
                self.iteration_count = 0

        # Decode translation
        translation = self.tokenizer.decode(
            decoder_input_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        return translation

In [ ]:
# Initialize decoder
online_decoder = OnlineSpeculativeDecoder()

# Example translation
source_text = "In a world where technology evolves at an unprecedented pace, individuals and organizations must adapt quickly to the rapid advancements in artificial intelligence, machine learning, and automation, ensuring that ethical considerations, environmental sustainability, and equitable access to resources are prioritized to create a future that benefits all of humanity."

# Time the translation
start_time = time.time()
translation = online_decoder.translate(source_text)
end_time = time.time()

print(f"Source: {source_text}")
print(f"Translation: {translation}\n")
print(f"Time taken: {end_time - start_time:.2f} seconds")

# Online V2

In [7]:
class OnlineSpeculativeDecoder:
    def __init__(
        self,
        target_model_name = "google-t5/t5-large",
        draft_model_name = "google-t5/t5-small",
        device = "cuda" if torch.cuda.is_available() else "cpu",
        gamma = 4,
        update_interval=2,  # Update draft model after every `update_interval` iterations
        buffer_size_threshold=2,  # Buffer size threshold for updates
        time_threshold=2,  # Time threshold (in seconds) for updates
    ):
        self.device = device
        self.gamma = gamma
        self.update_interval = update_interval
        self.buffer_size_threshold = buffer_size_threshold
        self.time_threshold = time_threshold
        self.last_update_time = time.time()  # Track last update time

        self.tokenizer = T5Tokenizer.from_pretrained(target_model_name)
        self.target_model = T5ForConditionalGeneration.from_pretrained(target_model_name).to(device)
        self.draft_model = T5ForConditionalGeneration.from_pretrained(draft_model_name).to(device)

        self.target_model.eval()
        self.draft_model.eval()

        # Buffers for storing token proposals and updates
        self.replay_buffer = []
        self.temp_buffer = []  # Temporary buffer for a single request

        # Counter for iteration tracking
        self.iteration_count = 0

    def get_draft_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        gamma: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get draft logits for gamma tokens"""
        draft_tokens = []
        draft_probs = []
        current_decoder_ids = decoder_input_ids

        # Generate gamma tokens from the draft model
        for _ in range(gamma):
            with torch.no_grad():
                outputs = self.draft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=current_decoder_ids,
                    return_dict=True
                )
                logits = outputs.logits[:, -1, :]  # Get logits for last position
                probs = F.softmax(logits, dim=-1)

                # Sample token
                token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
                prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)

                draft_tokens.append(token_id.item())
                draft_probs.append(prob.item())

                # Update decoder inputs for next iteration
                current_decoder_ids = torch.cat(
                    [current_decoder_ids, token_id.view(1, 1)],
                    dim=1
                )

                if token_id.item() == self.tokenizer.eos_token_id:
                    break

        return draft_tokens, draft_probs, current_decoder_ids, outputs.logits

    def get_target_probs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        draft_tokens: torch.Tensor
    ) -> torch.Tensor:
        """Get target probabilities for the draft tokens in parallel."""
        with torch.no_grad():
            # Add draft tokens to decoder input
            full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)

            outputs = self.target_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=full_decoder_ids,
                return_dict=True
            )

            # Get probabilities for positions before each draft token
            logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
            target_probs = F.softmax(logits, dim=-1)

            return target_probs.squeeze(0), outputs.logits
        
    def get_logits(self, model, input_ids, attention_mask, decoder_input_ids):
        return model(
            input_ids=input_ids,
            decoder_input_ids=decoder_input_ids,
            attention_mask=attention_mask,
        ).logits

    def verify_tokens(
        self,
        target_probs: torch.Tensor,
        draft_tokens: torch.Tensor,
        draft_probs: torch.Tensor,
    ) -> int:
        """Determine number of accepted tokens"""
        # Get target probabilities for the draft tokens
        target_probs_draft_tokens = target_probs.gather(
            -1,
            draft_tokens.unsqueeze(-1)
        ).squeeze(-1)

        # Calculate acceptance ratios
        acceptance_ratios = target_probs_draft_tokens / draft_probs

        # Sample uniform random numbers
        random_nums = torch.rand_like(acceptance_ratios)

        # Find number of accepted tokens
        # Accept if random number < min(1, target_prob / draft_prob)
        accepts = random_nums < torch.minimum(
            torch.ones_like(acceptance_ratios),
            acceptance_ratios
        )

        # Find first rejection
        try:
            n_accepted = torch.where(~accepts)[0][0].item()
        except:
            n_accepted = len(accepts)

        return n_accepted

        # accepted_tokens = []
        # for i in range(len(draft_tokens)):
        #     if target_probs[i] / draft_probs[i] > torch.rand(1).item():
        #         accepted_tokens.append(draft_tokens[i])
        #     else:
        #         break # Stop if token is not accepted

        # return len(accepted_tokens)
    
    # TODO: verify this, might need to do some window size thing
    # def update_draft_model(self):
    #     """Update draft model with the replay buffer."""
    #     if len(self.replay_buffer) == 0:
    #         return

    #     # Get draft tokens, draft and target probabilities from the replay buffer
    #     draft_tokens = torch.tensor([x[0] for x in self.replay_buffer], device=self.device)
    #     target_logits = torch.tensor([x[1] for x in self.replay_buffer], device=self.device)

    #     encoder_inputs = s
    #     output = self.draft_model(
    #         input_ids=encoder_inputs.input_ids,
    #         attention_mask=encoder_inputs.attention_mask,
    #         decoder_input_ids=decoder_input_ids,
    #         return_dict=True
    #     )

    def pad_to_2d(self, tensor_list, pad_token_id, max_len=None):
        if not isinstance(tensor_list[0], torch.Tensor):
            tensor_list = [torch.tensor(t).reshape(1, -1) for t in tensor_list]
        if max_len is None:
            max_len = max([t.shape[-1] for t in tensor_list])
        assert max_len > 0

        # Pad each tensor to the max length and stack them to form a 2D tensor
        result = torch.cat(
            [
                torch.nn.functional.pad(
                    tensor, (0, max_len - tensor.shape[-1]),
                    value=pad_token_id
                )
                for tensor in tensor_list
            ],
            dim=0
        )
        return result
        

    def soft_cross_entropy(self, predicts, targets, padding_mask=None):
        predict_log_prob = torch.nn.functional.log_softmax(predicts, dim=-1)
        targets_prob = torch.nn.functional.softmax(targets, dim=-1)
        entropy = -targets_prob * predict_log_prob
        expand_mask = padding_mask.unsqueeze(-1).expand_as(entropy)
        entropy.masked_fill_(expand_mask, 0)
        mean_entropy = entropy.sum() / (~padding_mask).sum()
        return mean_entropy

    def translate_dataset(
        self,
        sentences: List[str],
        max_length: int = 128
    ) -> List[str]:
        """Translate dataset using online speculative decoding."""

        self.iteration_count = 0
        self.replay_buffer = []

        translated_data = []

        for source_text in sentences:
            # Encode source text
            encoder_inputs = self.tokenizer(
                f"translate English to German: {source_text}",
                return_tensors="pt",
                padding=True
            ).to(self.device)

            # Initialize with start token
            decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)
            self.temp_buffer = []

            while decoder_input_ids.shape[1] < max_length:

                # Check for end of sequence
                if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                    break

                # Get draft tokens autoregressively
                # print("Encoder Inputs", encoder_inputs.input_ids.shape)
                draft_tokens, draft_probs, proposed_decoder_ids, draft_logits = self.get_draft_logits(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    self.gamma
                )

                draft_tokens = torch.tensor(draft_tokens, device=self.device)
                draft_probs = torch.tensor(draft_probs, device=self.device)

                if len(draft_tokens) == 0:
                    break

                # Get target probabilities in parallel
                target_probs, target_logits = self.get_target_probs(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    draft_tokens
                )
                # print(draft_logits.shape, target_logits.shape)

                # Verify tokens
                n_accepted = self.verify_tokens(target_probs, draft_tokens, draft_probs)

                # Accept verified tokens
                if n_accepted > 0:
                    decoder_input_ids = torch.cat([
                        decoder_input_ids,
                        draft_tokens[:n_accepted].unsqueeze(0)
                    ], dim=1)

                with torch.no_grad():
                    # print(target_logits.shape, draft_logits.shape)
                    probs = target_logits[:, -1, :] #- draft_logits[:, -1, :]
                    probs = F.softmax(probs, dim=-1)
                    token_id = torch.multinomial(probs, num_samples=1)
                    # print(probs.shape, token_id.shape)
                    decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)
                
                # rejected_tokens = draft_tokens[n_accepted]
                if n_accepted < len(draft_tokens):

                    self.temp_buffer.append((encoder_inputs, decoder_input_ids, target_logits, n_accepted))

            self.replay_buffer.extend(self.temp_buffer)
            self.iteration_count += 1

            if self.iteration_count % self.update_interval == 0:
                # self.update_draft_model()
                self.draft_model.train()
                
                # finetune over collected tokens and logits
                encoder_input_ids = self.pad_to_2d([x[0].input_ids for x in self.replay_buffer], 0)
                encoder_attention_mask = torch.stack([x[0].attention_mask[0] for x in self.replay_buffer], dim=0)
                decoder_input_ids = self.pad_to_2d([x[1] for x in self.replay_buffer], 0, 512)

                print(encoder_input_ids.shape, encoder_attention_mask.shape, decoder_input_ids.shape)

                target_logits = [x[2] for x in self.replay_buffer]
                for i in range(len(target_logits)):
                    temp = torch.zeros(1, 32128, device=self.device).repeat(512 - target_logits[i].shape[1], 1).unsqueeze(0)
                    target_logits[i] = torch.cat([target_logits[i], temp], dim=1)

                n_accepted_tokens = [x[3] for x in self.replay_buffer]

                # CUDA out of memory error
                draft_logits = self.get_logits(self.draft_model, encoder_input_ids, encoder_attention_mask, decoder_input_ids).float()

                # need to get loss only using the wrong tokens
                # TODO: check if we need to ignore the pad tokens in the mask
                mask = torch.ones_like(decoder_input_ids, dtype=torch.bool)
                for i in range(len(n_accepted_tokens)):
                    mask[i, n_accepted_tokens[i]:] = False
                loss = self.soft_cross_entropy(draft_logits, target_logits, mask)
                loss.backward()

                self.draft_model.eval()
                self.replay_buffer = []
                self.iteration_count = 0

            # Decode translation
            translation = self.tokenizer.decode(
                decoder_input_ids[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            translated_data.append(translation)
        return translated_data

In [8]:
# Initialize decoder
online_decoder = OnlineSpeculativeDecoder()

# Example translation
source_text = "In a world where technology evolves at an unprecedented pace, individuals and organizations must adapt quickly to the rapid advancements in artificial intelligence, machine learning, and automation, ensuring that ethical considerations, environmental sustainability, and equitable access to resources are prioritized to create a future that benefits all of humanity."
sents = [source_text] * 10
# Time the translation
start_time = time.time()
translation = online_decoder.translate_dataset(sents)
end_time = time.time()

for i, sent in enumerate(sents):
    print(f"Source: {sent}")
    print(f"Translation: {translation[i]}\n")
print(f"Time taken: {end_time - start_time:.2f} seconds")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

torch.Size([28, 69]) torch.Size([28, 69]) torch.Size([28, 512])


OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB. GPU 0 has a total capacity of 10.75 GiB of which 40.50 MiB is free. Including non-PyTorch memory, this process has 10.71 GiB memory in use. Of the allocated memory 9.81 GiB is allocated by PyTorch, and 111.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)